In [1]:
import requests
import datetime
import time
import pandas as pd

In [5]:
page_id = "crazyck101"

app_id = "393833577650615"
app_secret = "3ab280dac70995e4d8e601cbc025b53a"

access_token = app_id + "|" + app_secret

In [6]:
# 判斷response有無正常 正常 200，若無隔五秒鐘之後再試
def request_until_succeed(url):
    success = False
    while success is False:
        try: 
            req = requests.get(url)
            if req.status_code == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
            print("Error for URL %s: %s" % (url, datetime.datetime.now()))
            print("Retrying.")

    return req

# 取得Facebook data
def getFacebookPageFeedData(page_id, access_token, num_statuses):

    # Construct the URL string; see http://stackoverflow.com/a/37239851 for
    # Reactions parameters
    base = "https://graph.facebook.com/v2.8"
    node = "/%s/posts" % page_id 
    fields = "/?fields=message,link,created_time,type,name,id," + \
            "comments.limit(0).summary(true),shares,reactions" + \
            ".limit(0).summary(true)"
    parameters = "&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + fields + parameters

    # 取得data
    data = request_until_succeed(url).json()
    return data

# 取得該篇文章的 reactions like,love,wow,haha,sad,angry數目
def getReactionsForStatus(status_id, access_token):

    # See http://stackoverflow.com/a/37239851 for Reactions parameters
        # Reactions are only accessable at a single-post endpoint

    base = "https://graph.facebook.com/v2.6"
    node = "/%s" % status_id
    reactions = "/?fields=" \
            "reactions.type(LIKE).limit(0).summary(total_count).as(like)" \
            ",reactions.type(LOVE).limit(0).summary(total_count).as(love)" \
            ",reactions.type(WOW).limit(0).summary(total_count).as(wow)" \
            ",reactions.type(HAHA).limit(0).summary(total_count).as(haha)" \
            ",reactions.type(SAD).limit(0).summary(total_count).as(sad)" \
            ",reactions.type(ANGRY).limit(0).summary(total_count).as(angry)"
    parameters = "&access_token=%s" % access_token
    url = base + node + reactions + parameters

    # 取得data
    data = request_until_succeed(url).json()
    return data

def processFacebookPageFeedStatus(status, access_token):

    # 要去確認抓到的資料是否為空
    status_id = status['id']
    status_type = status['type']
    if 'message' not in status.keys():
        status_message = ''
    else:
        status_message = status['message']
    if 'name' not in status.keys():
        link_name = ''
    else:
        link_name = status['name']
    link = status_id.split('_')
    
    # 此連結可以回到該臉書上的post
    status_link = 'https://www.facebook.com/'+link[0]+'/posts/'+link[1]

    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    # 根據所在時區 TW +8
    status_published = status_published + datetime.timedelta(hours=8)
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') 
    
    # 要去確認抓到的資料是否為空
    if 'reactions' not in status:
        num_reactions = 0
    else:
        num_reactions = status['reactions']['summary']['total_count']
    if 'comments' not in status:
        num_comments = 0
    else:
        num_comments = status['comments']['summary']['total_count']
    if 'shares' not in status:
        num_shares = 0
    else:
        num_shares = status['shares']['count']

    def get_num_total_reactions(reaction_type, reactions):
        if reaction_type not in reactions:
            return 0
        else:
            return reactions[reaction_type]['summary']['total_count']
    
    # 取得該篇文章的 reactions like,love,wow,haha,sad,angry數目
    reactions = getReactionsForStatus(status_id, access_token)
    
    num_loves = get_num_total_reactions('love', reactions)
    num_wows = get_num_total_reactions('wow', reactions)
    num_hahas = get_num_total_reactions('haha', reactions)
    num_sads = get_num_total_reactions('sad', reactions)
    num_angrys = get_num_total_reactions('angry', reactions)
    num_likes = get_num_total_reactions('like', reactions)

    # 回傳tuple形式的資料
    return (status_id, status_message, link_name, status_type, status_link,
            status_published, num_reactions, num_comments, num_shares,
            num_likes, num_loves, num_wows, num_hahas, num_sads, num_angrys)


def scrapeFacebookPageFeedStatus(page_id, access_token):
    # all_statuses 用來儲存的list,先放入欄位名稱
    all_statuses = [('status_id', 'status_message', 'link_name', 'status_type', 'status_link',
            'status_published', 'num_reactions', 'num_comments', 'num_shares',
            'num_likes', 'num_loves', 'num_wows', 'num_hahas', 'num_sads', 'num_angrys')]
    
    has_next_page = True 
    num_processed = 0   # 計算處理多少post
    scrape_starttime = datetime.datetime.now()

    print("Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime))

    statuses = getFacebookPageFeedData(page_id, access_token, 100)

    while has_next_page:
        for status in statuses['data']:

            # 確定有 reaction 再把結構化後的資料存入 all_statuses
            if 'reactions' in status:
                all_statuses.append(processFacebookPageFeedStatus(status,access_token))

            # 觀察爬取進度,每處理100篇post,就輸出時間,
            num_processed += 1
            if num_processed % 100 == 0:
                print("%s Statuses Processed: %s" % (num_processed, datetime.datetime.now()))

        # 每超過100個post就會有next,可以從next中取得下100篇, 直到沒有next
        if 'paging' in statuses.keys():
            statuses = request_until_succeed(statuses['paging']['next']).json()
        else:
            has_next_page = False

    print("\nDone!\n%s Statuses Processed in %s" % \
        (num_processed, datetime.datetime.now() - scrape_starttime))
    
    return all_statuses




In [7]:
all_statuses = scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping crazyck101 Facebook Page: 2017-03-20 10:51:29.174158

100 Statuses Processed: 2017-03-20 10:52:03.116243
200 Statuses Processed: 2017-03-20 10:52:38.390855
300 Statuses Processed: 2017-03-20 10:53:12.885561
400 Statuses Processed: 2017-03-20 10:53:47.593162
500 Statuses Processed: 2017-03-20 10:54:22.654553

Done!
501 Statuses Processed in 0:02:54.503107


In [8]:
df = pd.DataFrame(all_statuses[1:], columns=all_statuses[0])

In [9]:

df.head()

,status_id,status_message,link_name,status_type,status_link,status_published,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys
0,1021581417961885_1225501984236493,【0319】卡提諾狂新聞 #057\n\n● 重複留言絕對無法抽獎！\n \n※ 第四名由於...,【0319】卡提諾狂新聞 #057,video,https://www.facebook.com/1021581417961885/post...,2017-03-19 19:53:24,50386,8165,10206,41992,329,132,7877,17,39
1,1021581417961885_1225484724238219,20：00 準時更新,,status,https://www.facebook.com/1021581417961885/post...,2017-03-19 19:21:36,7415,265,18,6457,28,56,373,11,490
2,1021581417961885_1225318954254796,我知道你們在等，一有消息我就告訴你們,Timeline Photos,photo,https://www.facebook.com/1021581417961885/post...,2017-03-19 14:54:44,19937,577,38,16649,29,108,1697,39,1415
3,1021581417961885_1224135674373124,委員 請問公蝦小嗎？ #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lr...,20170317中天新聞 繼林岱樺中油案 綠營再爆陳亭妃涉關說｜卡提諾狂新聞,link,https://www.facebook.com/1021581417961885/post...,2017-03-18 11:22:59,5716,337,276,4421,11,78,741,8,457
4,1021581417961885_1223387294447962,📣小助手來公佈【狂抽猛送 #5】得獎者名單\n本次獎項真的多到不.手.軟\n如果你是以下得...,卡提諾狂新聞,video,https://www.facebook.com/1021581417961885/post...,2017-03-17 17:09:01,986,75,0,910,6,4,4,57,5


In [11]:
path = page_id+'_post.csv'
df.to_csv(path,index=False,encoding='utf8')